In [ ]:
import os
import time
import random
import warnings
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.linalg import orth
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from sklearn.decomposition import NMF
from matplotlib.ticker import LogFormatter
from sklearn.preprocessing import MinMaxScaler
np.random.seed(29)
warnings.filterwarnings("ignore")

In [ ]:
def generate_action_set(trials, M, d, k):
    
    user_artists = pd.read_csv('../user_artists.dat', sep='\t')

    artist_counts = user_artists['artistID'].value_counts()
    artists_filtered = artist_counts[artist_counts >= 30].index
    user_artists_filtered = user_artists[user_artists['artistID'].isin(artists_filtered)]

    user_counts = user_artists_filtered['userID'].value_counts()
    users_filtered = user_counts[user_counts >= 30].index
    user_artists_final = user_artists_filtered[user_artists_filtered['userID'].isin(users_filtered)]

    rating_matrix = pd.pivot_table(user_artists_final, index = 'userID', columns = 'artistID', aggfunc = lambda x: x, fill_value = 0)

    scaler = MinMaxScaler()
    rating_matrix = pd.DataFrame(scaler.fit_transform(rating_matrix), columns=rating_matrix.columns)

    model = NMF(n_components = int(np.sqrt(d)), init = 'nndsvda', max_iter = 10000)
    W = model.fit_transform(rating_matrix)
    H = model.components_
    
    theta_true = np.eye(int(np.sqrt(d))).ravel()
    Theta = np.tile(theta_true, (M, 1)).T[np.newaxis, :]
    
    kmeans = KMeans(n_clusters = M, random_state = 0).fit(H.T)
    labels = kmeans.labels_
    
    Action_list = np.empty((1, W.shape[0], M), dtype = object)

    for t in range(W.shape[0]):

        for i in range(M):

            size = H[:, labels == i].shape[1]
            Action_list[0, t, i] = np.empty((size, d), dtype = object)

            for ac in range(size):

                temp = np.outer(W[t], H[:, labels == i][:, ac])
                Action_list[0, t, i][ac] = temp.ravel()
                
    return Action_list, Theta

In [ ]:
def init_altgdmin(X, t, Y, C, M, d, k):
    
    alpha = C * np.linalg.norm(Y, 'fro') / np.sqrt(t * M)
    Y_trunc = np.where(np.abs(Y) > alpha, 0, Y)
    
    Theta_0 = np.zeros((d, M))
    
    for i in range(M):
        
        Theta_0[:, i] = np.sum(X[i].dot(np.diag(Y_trunc[i])), axis=1) / t
    
    U_0, Sigma_0, V_0 = np.linalg.svd(Theta_0, full_matrices = False)
    B_hat = U_0[:, :k]
    S_k = np.zeros((k, k))
    np.fill_diagonal(S_k, Sigma_0[:k])
    
    return B_hat

In [ ]:
def AltGD_Min(X, Y, M, k, B_hat):
    
    W_hat = np.zeros((k, M))
    
    for i in range(M):

        W_hat[:, i] = np.linalg.lstsq(np.dot(X[i].T, B_hat), Y[i], rcond = None)[0]
    
    return W_hat

In [ ]:
CR_Trials_ICML = []
CR_Trials_E2TC = []
CR_Trials_Naive = []
d = 100
M = 10
k = 1
C = 4
R = 1
ld = 1
ld_0 = 1
trials = 1
sigma = 1e-3
delta = 1e-3
epoch_iter = 300
iterations = 1000
every_point = 25
A, Theta = generate_action_set(trials, M, d, k)

In [ ]:
# generate the noise of reward for each iteration
noise_list = np.random.normal(0, sigma ** 2, size = (trials, iterations, M))

In [ ]:
action_exploration_list = []

for t in range(epoch_iter):
    
    temp = []

    for i in range(M):

        action = np.random.choice(A[0][t, i].shape[0])
        temp.append(action)
        
    action_exploration_list.append(temp)

In [ ]:
Context_idx = np.random.choice(A.shape[1], size = iterations)

In [ ]:
# ICML
start = time.time()

for T in range(trials):
    
#     print('Trial:', T)
    
    action_set = A[T]
    noise_T = noise_list[T]
    
    B_hat = np.zeros((d, k))
    W_hat = np.zeros((k, M))
    Theta_hat = B_hat.dot(W_hat)
    
    cumulative_regret = 0
    cummulative_regret_list = [0]
    
    X = [np.empty((d, epoch_iter)) for _ in range(M)]
    Y = [np.empty(epoch_iter) for _ in range(M)]
    
    for t in tqdm(range(epoch_iter), desc = 'exploration iterations'):
        
        for i in range(M):
            
            action = action_exploration_list[t][i]
            reward = action_set[t, i][action].dot(Theta[T][:, i])
            cumulative_regret += np.max(action_set[t, i].dot(Theta[T][:, i])) - reward
            
            X[i][:, t] = action_set[t, i][action].reshape(-1)
            Y[i][t] = reward + noise_T[t, i]
            
        cummulative_regret_list.append(cumulative_regret)
        
    B_hat = init_altgdmin(X, epoch_iter, Y, C, M, d, k)
    W_hat = AltGD_Min(X, Y, M, k, B_hat)
    Theta_hat = B_hat.dot(W_hat)
    
    for t in tqdm(range(epoch_iter, iterations), desc = 'commit iterations'):
        
        for i in range(M):
            
            context = Context_idx[t]
            
            action = np.argmax(np.dot(action_set[context, i], Theta_hat[:, i]))
            reward = action_set[context, i][action].dot(Theta[T][:, i])
            cumulative_regret += np.max(action_set[context, i].dot(Theta[T][:, i])) - reward
            
#             action = np.argmax(np.dot(action_set[t, i], Theta_hat[:, i]))
#             reward = action_set[t, i][action].dot(Theta[T][:, i])
#             cumulative_regret += np.max(action_set[t, i].dot(Theta[T][:, i])) - reward
            
        cummulative_regret_list.append(cumulative_regret)
        
#         print('t: {} regret: {} error: {}:'.format(num, regret, error))

    CR_Trials_ICML.append((T, cummulative_regret_list))
    
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# INFINITE-ACTION SETTING
start = time.time()

for T in range(trials):
    
#     print('Trial:', T)
    
    action_set = A[T]
    noise_T = noise_list[T]
    
    B_hat = np.zeros((d, k))
    W_hat = np.zeros((k, M))
    Theta_hat = B_hat.dot(W_hat)
    
    cumulative_regret = 0
    cummulative_regret_list = [0]
    
    M_hat = np.zeros((d, d))
    
    X = [np.empty((d, epoch_iter)) for _ in range(M)]
    Y = [np.empty(epoch_iter) for _ in range(M)]
    
    for t in tqdm(range(epoch_iter), desc = 'exploration iterations'):
        
        for i in range(M):
            
            action = action_exploration_list[t][i]
            reward = action_set[t, i][action].dot(Theta[T][:, i])
            cumulative_regret += np.max(action_set[t, i].dot(Theta[T][:, i])) - reward
            
            X[i][:, t] = action_set[t, i][action].reshape(-1)
            Y[i][t] = reward + noise_T[t, i]
            
            M_hat = M_hat + (reward + noise_T[t, i]) ** 2 * np.dot(action_set[t, i][action].reshape(-1, 1), action_set[t, i][action].reshape(1, -1))
            
        cummulative_regret_list.append(cumulative_regret)
            
    M_hat = M_hat / (epoch_iter * M)
    M_hat = M_hat.astype(np.float64)
    U, Sigma, V = np.linalg.svd(M_hat, full_matrices=False)
    B_hat = U[:, :k]
    
    for i in range(M):
        
        W_hat[:, i] = np.linalg.lstsq(np.dot(X[i].T, B_hat), Y[i], rcond = None)[0]
        
    Theta_hat = B_hat.dot(W_hat)
    
    for t in tqdm(range(epoch_iter, iterations), desc = 'commit iterations'):
        
        for i in range(M):
            
            context = Context_idx[t]
            
            action = np.argmax(np.dot(action_set[context, i], Theta_hat[:, i]))
            reward = action_set[context, i][action].dot(Theta[T][:, i])
            cumulative_regret += np.max(action_set[context, i].dot(Theta[T][:, i])) - reward
            
#             action = np.argmax(np.dot(action_set[t, i], Theta_hat[:, i]))
#             reward = action_set[t, i][action].dot(Theta[T][:, i])
#             cumulative_regret += np.max(action_set[t, i].dot(Theta[T][:, i])) - reward
            
        cummulative_regret_list.append(cumulative_regret)
        
#         print('t: {} regret: {} error: {}:'.format(num, regret, error))

    CR_Trials_E2TC.append((T, cummulative_regret_list))
    
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
# Naive Approach
start = time.time()

for T in range(trials):
    
#     print('Trial:', T)
    
    action_set = A[T]
    noise_T = noise_list[T]
    
    W = np.zeros((M, d, d))
    U = np.zeros((M, d, 1))
    
    Theta_hat = np.zeros((d, M))
    delta_prime = delta / (4 * iterations)
    
    cumulative_regret = 0
    cummulative_regret_list = [0]
    
    for t in tqdm(range(iterations), desc = 'iterations'):
        
        for i in range(M):
            
            # sample eta
            eta = np.random.multivariate_normal(mean = np.zeros(d), cov = np.eye(d))
            
            # compute RLS-estimate theta_hat and V
            V = ld * np.eye(d) + W[i]
            Theta_hat[:, i] = np.dot(np.linalg.inv(V), U[i]).reshape(-1)
            
            # compute the beta
            beta = R * np.sqrt(2 * np.log((ld + t) ** (d / 2) * ld ** (- d / 2) / delta_prime)) + np.sqrt(ld) * np.linalg.norm(Theta_hat[:, i])
            
            # calculate V^{-1/2}
            eigvals, eigvecs = np.linalg.eigh(V)
            temp = eigvecs.dot(np.diag(1.0 / np.sqrt(eigvals))).dot(eigvecs.T)
            
            # compute theta_tilde
            theta_tilde = Theta_hat[:, i] + beta * (temp.dot(eta)).reshape(-1)
            
            if t < epoch_iter:
                
                context = t
                
            else:
                
                context = Context_idx[t]
            
            action = np.argmax(np.dot(action_set[context, i], theta_tilde))
            feature = action_set[context, i][action].astype(np.float64)
            reward = feature.dot(Theta[T][:, i])
            cumulative_regret += np.max(action_set[context, i].dot(Theta[T][:, i])) - reward
            
#             action = np.argmax(np.dot(action_set[t, i], theta_tilde))
#             feature = action_set[t, i][action].astype(np.float64)
#             reward = feature.dot(Theta[T][:, i])
#             cumulative_regret += np.max(action_set[t, i].dot(Theta[T][:, i])) - reward
            
            W[i] += np.dot(feature.reshape(-1, 1), feature.reshape(1, -1))
            U[i] += feature.reshape(-1, 1) * (reward + noise_T[t, i])
            
        cummulative_regret_list.append(cumulative_regret)
        
#         print('t: {} regret: {} error: {}:'.format(num, regret, error))

    CR_Trials_Naive.append((T, cummulative_regret_list))
    
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
def prepare_plot_data(CR_Trials, iterations, trials):
    
    x_value = np.array([i for i in range(iterations + 1)])
    y_value = np.zeros(iterations + 1)
    
    for i in range(iterations + 1):
        
        for T in range(trials):
            
            y_value[i] += CR_Trials[T][1][i]
            
    y_value = y_value / trials
            
    return x_value, y_value

In [ ]:
x_proposed, y_proposed = prepare_plot_data(CR_Trials_ICML, iterations, trials)
x_E2TC, y_E2TC = prepare_plot_data(CR_Trials_E2TC, iterations, trials)
x_Naive, y_Naive = prepare_plot_data(CR_Trials_Naive, iterations, trials)

In [ ]:
plt.plot(x_proposed, y_proposed, color = 'r')
plt.plot(x_E2TC, y_E2TC, color = 'b')
plt.plot(x_Naive, y_Naive, color = 'y')

In [ ]:
np.save('CR_Trials_ICML.npy', np.array(CR_Trials_ICML, dtype=object))
np.save('CR_Trials_E2TC.npy', np.array(CR_Trials_E2TC, dtype=object))
np.save('CR_Trials_Naive.npy', np.array(CR_Trials_Naive, dtype=object))